In [1]:
import os,numpy as np, cv2 as cv2 , tensorflow as tf,time
import face_recognition
from tensorflow.python.keras.utils.data_utils import get_file
import mediapipe as mp 

known_faces_dir="data/known"
unknown_faces_dir="data/unknown"

tolerance=0.6
frame_thickness=3
font_thickness=2
MODEL="cnn"

known_faces=[]
known_names=[]
match=''
np.random.seed(20)

In [2]:
from PIL import ImageTk, Image
class Detector:
    def __init__(self):
        pass
    
    def readClasses(self,classesFilePath):
        with open(classesFilePath,'r') as f:
            self.classesList=f.read().splitlines()
            self.colorsList=np.random.uniform(low=0,high=255,size=(len(self.classesList),3))
    def  DownloadModel(self,modelUrl):
        filename=os.path.basename(modelUrl)
        self.modelName=filename[: filename.index('.')]
        self.cacheDir="./pretrained_models"
        os.makedirs(self.cacheDir,exist_ok=True)
        get_file(fname=filename,origin=modelUrl,cache_dir=self.cacheDir,cache_subdir="checkpoints",extract=True)

    def LoadModel(self,modelPath):      
        self.model=tf.saved_model.load(os.path.join(self.cacheDir,"checkpoints", self.modelName,"saved_model"))

        print("the model loaded .......")
    def DrawBoundingBox(self,image,threshold):
        inputTensor=cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB)             # convert color of image to  RGB
        inputTensor= tf.convert_to_tensor(inputTensor,dtype=tf.uint8)                   # flatten the input , convert the image to tensor
        inputTensor=inputTensor[tf.newaxis,...]
        # -----------------------------------------------------
        detections=self.model(inputTensor)
        boxs=detections['detection_boxes'][0].numpy()                                         # define all boxes that found on the image
        classIndexs = detections['detection_classes'][0].numpy().astype(np.int32)
        classScores = detections['detection_scores'][0].numpy()
        # ------------------------------------------------------------------
        imageHeight, imageWidth ,imc= image.shape
        bboxIds=tf.image.non_max_suppression(boxs, classScores,max_output_size=50,iou_threshold=threshold,score_threshold=threshold)
#         global NuPerson
#         NuPerson=0
        if len(bboxIds) !=0:
            for i in bboxIds:
                bbox= tuple(boxs[i].tolist())
                classConfidence = round(100* classScores[i])
                classIndex = classIndexs[i]
                if (classIndex == 2):
                    self.msg="object is found"
                else:
                    self.msg="object is not found"
                classLabel = self.classesList[classIndex].upper()
                classColor = self.colorsList[classIndex]
                displayText = "{}: {}%".format(classLabel, classConfidence)
        #         ----------------------------------------------->
                ymin, xmin, ymax,xmax = bbox
                xmin,xmax, ymin, ymax = (xmin*imageWidth, xmax* imageWidth, ymin * imageHeight, ymax*imageHeight)
                xmin, xmax, ymin, ymax = int(xmin), int(xmax), int(ymin), int(ymax)
                lineWidth = min(int((xmax - xmin) * 0.3), int((ymax - ymin) * 0.3))
                theckNess = 2
                cv2.line(image, (xmin, ymin), (xmin + lineWidth, ymin), classColor, thickness=theckNess)
                cv2.line(image, (xmin, ymin), (xmin, ymin + lineWidth), classColor, thickness=theckNess)

                cv2.line(image, (xmax, ymin), (xmax - lineWidth, ymin), classColor, thickness=theckNess)
                cv2.line(image, (xmax, ymin), (xmax, ymin + lineWidth), classColor, thickness=theckNess)

                cv2.line(image, (xmin, ymax), (xmin + lineWidth, ymax), classColor, thickness=theckNess)
                cv2.line(image, (xmin, ymax), (xmin, ymax - lineWidth), classColor, thickness=theckNess)
                cv2.line(image, (xmax, ymax), (xmax - lineWidth, ymax), classColor, thickness=theckNess)
                cv2.line(image, (xmax, ymax), (xmax, ymax - lineWidth), classColor, thickness=theckNess)

#                 =====================================

                cv2.rectangle(image ,(xmin,ymin), (xmax, ymax),color=classColor, thickness=1)
                cv2.putText(image, displayText,(xmin, ymin - 10),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,classColor,1)
        return image
    def recognition_faces(self):
        for name in os.listdir(known_faces_dir):
            for filename in os.listdir(f"{known_faces_dir}/{name}"):
                image = face_recognition.load_image_file(f"{known_faces_dir}/{name}/{filename}")
                encoding = face_recognition.face_encodings(image)[0]
                known_faces.append(encoding)
                known_names.append(name)
    def DrawingFaceBoundingBox(self,img):
        mpFaceDetection = mp.solutions.face_detection
        FaceDetection = mpFaceDetection.FaceDetection()
        mpDraw = mp.solutions.drawing_utils
#         cv2.putText(img, "N. Person:" + str(NuPerson), (10, 20), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)        
        results = FaceDetection.process(img)
        if results.detections:
            for id, detection in enumerate(results.detections):
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, ic = img.shape
                bbox = int(bboxC.xmin * iw), int(bboxC.ymin * ih),\
                       int(bboxC.width * iw), int(bboxC.height * ih)
                img = self.fancyDraw(img,bbox)
        return img 
    # for drawing the bounding box      
    def fancyDraw(self, img, bbox, l=20, t=2,rt=1):
        x, y, w, h = bbox
        x1,y1 = x+w,y+h 
        cv2.rectangle(img, bbox, (0,255,255),rt)
       # courner 1 = TOP LEFT X,Y       
        cv2.line(img, (x,y),(x+l,y),(0,255,255),t)
        cv2.line(img, (x,y),(x,y+l),(0,255,255),t)
        # courner 2 = TOP RIGHT X1,Y
        cv2.line(img, (x1,y),(x1-l,y),(0,255,255),t)
        cv2.line(img, (x1,y),(x1,y+l),(0,255,255),t)
        
        # courner 3 = BOTTOM LEFT x,y1
        cv2.line(img, (x,y1),(x+l,y1),(0,255,255),t)
        cv2.line(img, (x,y1),(x,y1-l),(0,255,255),t)
        # courner 4 = BOTTOM RIGHT X1,Y1
        cv2.line(img, (x1,y1),(x1-l,y1),(0,255,255),t)
        cv2.line(img, (x1,y1),(x1,y1-l),(0,255,255),t)
        return img
    def DetectOnImage(self, imagePath,threshold):
        image=cv2.imread(imagePath)
        # ----------------------------------
        self.recognition_faces()
        locations = face_recognition.face_locations(image, model=self.model);
        encodings = face_recognition.face_encodings(image, locations);
        # landmark=face_recognition.face_landmarks(image,locations,model= MODEL)
        imageboxed = self.DrawBoundingBox(image, threshold)
        for face_encoding, face_location in zip(encodings, locations):
            rslt = face_recognition.compare_faces(known_faces, face_encoding, tolerance)
            match = None;
            if True in rslt:
                match = known_names[rslt.index(True)];
                cv2.putText(imageboxed, "Matched:" + str(match), (20, 70), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
            else:
                 match="Unknown"
            cv2.putText(imageboxed, "Matched:" + str(match), (20, 70), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
        # ---------------------------------
        cv2.imshow("result:",imageboxed)
        cv2.imwrite("detect"+'.jpg', imageboxed)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [3]:
classesFilePath="model_data/coco.names"
ModelUrl="hqqttp://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz"
modelPath="pretrained_model/"

imagePath="data/im2.jpg"
videoPath="data/eln.mp4"
img = "database/saidd/"
threshold=0.5
detector=Detector()


detector.readClasses(classesFilePath)
detector.DownloadModel(ModelUrl)
detector.LoadModel(modelPath)
#     detector.DetectOnImage(imagePath,threshold)
# detector.DetectOnVideo(videoPath ,threshold)

the model loaded .......


In [ ]:
from tkinter import *
from tkinter.ttk import * 
from tkinter.filedialog import askopenfile 
from tkvideo import tkvideo
import time
import datetime
from turtle import color, width 
from PIL import ImageTk, Image
import tkinter as tk
from tkinter import filedialog 
import cv2 as cv2
import os 
window = Tk()
window.title("Search for object")
window.geometry('1250x850')
window.resizable(0,0)
# create the left frame 
left_frame = tk.Frame(window,width=620,height=825)
left_frame.grid(row=0,column=1)

# create the top left frame 
top_left_frame = tk.Frame(left_frame,width=620,height=80)
top_left_frame.grid(row=0,column=1)
#  create elements of top left frame 
tk.Button(top_left_frame, text="Select Image", font="Helvetica  14 bold",bg="yellow" ,fg="black", width=15,command=lambda:upload_Images() ).grid(row=0,column=2,pady=5)

# create function of uploading images 
def upload_Images():
    global filename,img
    F_Types=[('PNG files','*.png'),('Jpg files','*.jpg'),('Jpeg files','*.jpeg')]
    filename = tk.filedialog.askopenfilename(filetypes=F_Types)

    img= Image.open(filename)
    img=img.resize((300,300))
    image = ImageTk.PhotoImage(img)
    
    e1=tk.Button(bot_top_2_right_frame,bg="red")
    e1.grid(row=1,column=0,padx=10)
    e1.image=image
    e1['image']=image
    if image :
        AddToDataBaseBtn()
    if image is not None:
        pass
# create the bottom left frame 
bottom_left_frame = tk.Frame(left_frame,width=620,height=745)
bottom_left_frame.config(background="yellow")
bottom_left_frame.grid(row=1,column=1)

#  create elements of bottom left frame 
f1 = tk.LabelFrame(bottom_left_frame,bg="red")
f1.grid(row=0,column=1)
l1= tk.Label(f1, bg="red")
l1.grid(row=1,column=1)

# load video 
# player = tkvideo("database/el1.mp4", l1,loop=1, size=(650,745))
# player.play()

# ===============================================


def DetectOnvideo():
    cap=cv2.VideoCapture("data/eln.mp4")
    if (cap.isOpened() == False):
        print("Error in openning the video ....")
        return
    (success, image)=cap.read()
    starttime=0
    while success:
        currenttime=time.time()
        fps = 1 / (currenttime - starttime)
        starttime = currenttime
        bbox=detector.DrawBoundingBox(image,threshold);
        detector.recognition_faces()
        img_RGB = cv2.cvtColor(bbox, cv2.COLOR_RGB2BGR)
        img2 = detector.DrawingFaceBoundingBox(img_RGB)
        locations = face_recognition.face_locations(image, model=MODEL);
        encodings = face_recognition.face_encodings(image, locations);
        match = None;
        global dateFound,Location,msgg
        for face_encoding, face_location in zip(encodings, locations):
            rslt = face_recognition.compare_faces(known_faces, face_encoding, tolerance)    

            
            if True in rslt:                                
                match = known_names[rslt.index(True)];    
                msgg=" MATCHED !"
                dateFound = datetime.datetime.now()
                Location = "Location of the camera !"
            else:
                msgg=None
                dateFound = None
                Location = None
            img2=cv2.resize(img2,(650,745),fx=0,fy=0,interpolation=cv2.INTER_CUBIC)
            bb = ImageTk.PhotoImage(Image.fromarray(img2),master=bottom_left_frame)
            l1['image'] =bb
            window.update()
        (success,image)=cap.read()
# ===============================================
# create the right frame 
right_frame = tk.Frame(window,width=620,height=825)
right_frame.grid(row=0,column=2)

# create the top right frame 
# top_right_frame = tk.Frame(right_frame,width=620,height=400)
# top_right_frame.grid(row=0,column=1)

top_right_frame = tk.LabelFrame(right_frame, bg="gray" ,width=620,height=400)
top_right_frame.grid(row=0,column=1)
top_right_frame.grid_propagate(False)
# create the top 2 right frame 
top_2_right_frame = tk.Frame(top_right_frame,width=620,height=60)
top_2_right_frame.grid(row=1,column=1)
# create the bottom top 2 right frame 
bot_top_2_right_frame = tk.Frame(top_right_frame,width=620,height=340,bg="gray")
bot_top_2_right_frame.grid(row=2,column=1, padx=20, pady=15)

#  create elements of top right frame  
tk.Label(top_2_right_frame, text="Searching person", font="Helvetica 14 bold").grid(row=0,column=0, pady=5,padx=10)
def AddToDataBaseBtn():
    btnsFrame= tk.Frame(bot_top_2_right_frame,bg="gray")
    btnsFrame.grid(row=1,rowspan=2,column=1)
    tk.Button(btnsFrame, text="Add Image", font="Helvetica  14 bold",bg="green" ,fg="white", width=15,command=lambda:AddToDataBase()).grid(row=1,column=2,padx=10)
    tk.Button(btnsFrame, text="Searching", font="Helvetica  14 bold", fg="white",bg="red", width=15,command=lambda:Search()).grid(row=2,column=2,padx=10 ,pady=10)   
# save the image     
def AddToDataBase():
    parent_dir= "data/known/"
    path = os.path.join(parent_dir,str(time.time()))
    mode=0o666
    os.mkdir(path, mode)
    timed = str(datetime.datetime.now().today()).replace(":"," ")+".jpg"
    img.save(f"{path}/{timed}")
    
# create the bottom  right frame 
bottom_right_frame = tk.Frame(right_frame,width=620,height=425)
bottom_right_frame.config(background="gray")
bottom_right_frame.grid(row=1,column=1)
# create the top bottom right frame 
top_bottom_right_frame = tk.Frame(bottom_right_frame,width=620,height=65)
top_bottom_right_frame.grid(row=0,column=1, pady=20)
# some elements 
tk.Label(top_bottom_right_frame, text="Information About Searching Person",fg="white",bg="grey", font="Helvetica 14 bold").grid(row=0,column=0)
# create the bottom bottom right frame 
bottom_bottom_right_frame = tk.Frame(bottom_right_frame ,bg="grey",width=620,height=360)
bottom_bottom_right_frame.grid(row=1,column=1)
# some elements 
bf = tk.LabelFrame(bottom_bottom_right_frame, bg="grey" ,width=620,height=360)
bf.grid(row=0,column=0)

def Search():
    bf2 = tk.LabelFrame(bf, bg="gray" ,width=620,height=360)
    bf2.grid(row=0,column=1,rowspan=3,sticky="E",ipadx=0,columnspan=1)
    bf2.grid_propagate(False)
    tk.Label(bf2, text=f"Match Found   : {msgg}",fg="red", font="Helvetica 14 bold").grid(row=1,column=0,padx=0,pady=13)
    tk.Label(bf2, text=f"Date Found    : {dateFound}", font="Helvetica 14 bold").grid(row=2,column=0,padx=0,pady=13)
    tk.Label(bf2, text=f"Location Found: {Location}", font="Helvetica 14 bold").grid(row=3,column=0,padx=0,pady=13)
DetectOnvideo()


window.mainloop()